### TODO: realizar generalização na copia do dataset e nao na referencia

In [1]:
import random as rdm
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("salario.csv")
df.head()

,id,pais,estado,cidade,genero,data,salario
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87


In [3]:
df.describe()

,id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


## Unindo os atributos de localidade

In [4]:
ids = df.id
cities = df.cidade
states = df.estado
countries = df.pais

In [5]:
locality = []
for i in range(len(cities)):
    locality.append([ids[i], str('{}, {}, {}'.format(cities[i], states[i], countries[i]))])

In [6]:
df_locality = pd.DataFrame(locality, columns=['id', 'localidade'])
df_locality.head()

,id,localidade
0,1,"Mobile, Alabama, United States"
1,2,"Rego, Porto, Portugal"
2,3,"Emmaboda, Kalmar, Sweden"
3,4,"Sala, Västmanland, Sweden"
4,5,"Quimperlé, Bretagne, France"


## Unindo os dataframes

In [7]:
join_dataset = pd.concat([df, df_locality], axis=1, join='inner')
join_dataset.head()

,id,pais,estado,cidade,genero,data,salario,id,localidade
0,1,United States,Alabama,Mobile,Male,10/01/1987,$5407.86,1,"Mobile, Alabama, United States"
1,2,Portugal,Porto,Rego,Male,29/11/2007,$7833.82,2,"Rego, Porto, Portugal"
2,3,Sweden,Kalmar,Emmaboda,Male,23/06/1984,$5530.85,3,"Emmaboda, Kalmar, Sweden"
3,4,Sweden,Västmanland,Sala,Female,04/11/1983,$2803.76,4,"Sala, Västmanland, Sweden"
4,5,France,Bretagne,Quimperlé,Female,18/11/1990,$2201.87,5,"Quimperlé, Bretagne, France"


## Removendo atributos

In [8]:
dataset = join_dataset.drop(columns = ['id', 'pais', 'estado', 'cidade'])
dataset.describe()

,genero,data,salario,localidade
count,1000,1000,1000,1000
unique,2,956,1000,769
top,Female,17/01/1981,$4430.56,"Stockholm, Stockholm, Sweden"
freq,542,3,1,15


## Função de generalização da data

In [9]:
def date_generalization(dataset, level):
    if level == 1:
        # date '**/MM/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**' + dataset.iloc[data]['data'][2:10])
    elif level == 2:
        # date '**/**/YYYY'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**' + dataset.iloc[data]['data'][5:10])
    elif level == 3:
        # date'**/**/****'
        for data in range(len(dataset)):
            dataset.iloc[data]['data'] = str('**/**/****')
        
    return dataset

In [10]:
def locality_generalization(dataset, level):
    if level == 0:
        # locality 'city'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = locais[0]+","+locais[1]+","+locais[2]
    elif level == 1:
        # locality 'state,'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,"+locais[1]+","+locais[2]
    elif level == 2:
        # locality 'country'
        for data in range(len(dataset)):
            locais = dataset.iloc[data]['localidade'].split(',')
            dataset.iloc[data]['localidade'] = "**,**,"+locais[2]
    elif level == 3:
        # locality '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['localidade'] = '**'
        
    return dataset

In [11]:
def gender_generalization(dataset, level):
    if level == 1:
        # gender '*'
        for data in range(len(dataset)):
            dataset.iloc[data]['genero'] = '*'
            
    return dataset

## Função de Precisão

$precision (D, Na, h, HGVA) = 1 - \frac{\sum_{i=1}^{Na} \sum_{j=1}^{|D|} \frac{h}{HGVA_i}}{|D| * Na}
$

- D: conjunto de dados
- Na: número de atributos semi-identificadores
- h: altura da hierarquia de generalização de valor do atributo Ai após anonimização
- |HGVAi|: altura máxima da hierarquia

In [12]:
def precision(D, Na, h, HGVA):
    summation = 0
    for i in range(Na):
        for j in range (D):
            summation += (h[j] / (HGVA[i]))
    
    return 1 - (summation / (D * Na))
            

In [13]:
def soma(x,y):
    print()

# Gerar dados K-anonimizados

In [17]:
def anonymize(df_faltante, k):

    d = {'data': [], 'genero': [], 'localidade':[], 'salario':[]}
    df_final = pd.DataFrame(data=d)

    Ordem_Generalizacoes = [(date_generalization, 1),
                            (locality_generalization, 1),
                            (date_generalization, 2),
                            (locality_generalization, 2),
                            (gender_generalization, 1),
                            (date_generalization, 3),
                            (locality_generalization, 3),
                            (soma, 1)]
    generalizações = []
    for generalizacao_Atula in Ordem_Generalizacoes:

        aux = df_faltante.groupby(["data","genero","localidade"])["localidade"].count()

        atributos_K_anonimizados = aux[aux >= k].index

        for i in atributos_K_anonimizados:
            data = i[0]
            genero = i[1]
            localidade = i[2]

            indices_prontos = df_faltante[ (df_faltante['data'] == data) & (df_faltante['genero'] == genero) & (df_faltante["localidade"] == localidade) ].index#[0]

            df_final = df_final.append( df_faltante.iloc[indices_prontos], ignore_index = True )
            df_faltante.drop(indices_prontos , inplace=True)
            df_faltante = df_faltante.reset_index(drop=True)

        generalizacao_Atula[0](df_faltante, generalizacao_Atula[1])
    
#     precision(df_final, 3, , [4, 1, 3])
    df_final.to_csv (r'output/export_dataframe_k_' + str(k) +'.csv', index = None, header=True)


In [18]:
for k in [2, 4, 8, 16]:
    anonymize(dataset.copy(), k)

FileNotFoundError: [Errno 2] No such file or directory: 'output/export_dataframe_k_2.csv'